In [1]:
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import Rectangle


In [2]:
data_dir = '/Users/tessarusch/Library/CloudStorage/Box-Box/COVID-19 Adolphs Lab/core_analysis/external_selfResp_combined'


In [3]:
selfResp_data = pd.read_csv(os.path.join(data_dir, 'CVD_core_summary_scores.csv'))
extMeas_data = pd.read_csv(os.path.join(data_dir, 'externalMeasures_withRestrictionSum_county.csv'))
wave_to_week =  pd.read_csv(os.path.join(data_dir, 'wave_to_week.csv'))
extMeas_data = extMeas_data.merge(wave_to_week[['wave','week']], on = ['wave', 'week'], how = 'left')


Demographics 

In [4]:
w1_data = selfResp_data.loc[selfResp_data.wave == 1].copy()
w1_data['race'].value_counts()/len(w1_data) *100

White                                        73.051225
Asian                                        10.690423
Black or African American                     7.405345
Multiracial                                   5.456570
Other                                         1.614699
Prefer not to disclose                        0.890869
American Indian/Alaska Native                 0.723831
Native Hawaiian or Other Pacific Islander     0.167038
Name: race, dtype: float64

In [5]:
w1_data.sex = w1_data.sex.replace({'Female, Female': 'Female', 'CONSENT REVOKED': 'Prefer not to say' })
w1_data['sex'].value_counts()/len(w1_data) *100

Female               50.501114
Male                 49.053452
Prefer not to say     0.445434
Name: sex, dtype: float64

In [6]:
w1_data['ethnicity'].value_counts()/len(w1_data) *100

Not Hispanic or Latino     89.253898
Hispanic or Latino          9.521158
 Prefer not to disclose     1.224944
Name: ethnicity, dtype: float64

In [7]:
w1_data['weekly_income_self_cat'].value_counts()/len(w1_data) *100

<$250            42.538976
$500 - $999      17.316258
$250 - $499      13.530067
$1000 -$1499     11.915367
$1500 - $2999     8.741648
>$3000            3.006682
Don't know        2.951002
Name: weekly_income_self_cat, dtype: float64

In [8]:
w1_data['education'].value_counts()/len(w1_data) *100

Bachelor's degree          32.349666
Some college               25.334076
Master's degree            12.026726
High school                11.080178
Associate's degree         10.968820
Some graduate education     3.786192
Professional degree         2.004454
PhD                         1.336303
Some high school            0.946548
Other                       0.167038
Name: education, dtype: float64

In [9]:
w1_data['polit_party'].value_counts()/len(w1_data) *100

Democrat       54.398664
Independent    27.004454
Republican     15.311804
Other           3.285078
Name: polit_party, dtype: float64

In [11]:
print(w1_data.age.mean())
print(w1_data.age.min())
print(w1_data.age.max())

37.299554565701555
18.0
82.0


In [12]:
w1_data = selfResp_data.loc[selfResp_data.wave == 1].copy()


In [13]:
data = selfResp_data.merge(extMeas_data, on = ['PROLIFIC_PID', 'wave','week'])
data = data.loc[data.wave_day == 'start_date-1', :]
data = data.loc[data.wave.isin([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18]),:].reset_index(drop = True)


In [25]:
# z-score data and add small random noise
idx_cols = ['PROLIFIC_PID','state', 'county', 'loc_description','wave', 'wave_day', 'date', 
            'state_county', 'sex', 'agecats','weekly_income_self_cat','max_social_health_impact',
            'spouse_health_impact', 'parent_health_impact','grandparent_health_impact', 
            'thirdDegree_health_impact','child_health_impact', 'friend_health_impact']
value_cols = ['safety_measures', 'cvd_concern','cvd_norm_agreement', 
             'restrict_f1', 'restrict_f2', 'restrict_f3',
             'cumsum_cases','cumsum_deaths','cases_avg', 'cases_avg_per_100k','deaths_avg', 'deaths_avg_per_100k', 
             'ueRate','restriction_sum']
data_zscore = data[idx_cols + value_cols].copy()
data_zscore[value_cols] = (data[value_cols] - data[value_cols].mean())/data[value_cols].std()
#data_zscore[value_cols] = data_zscore[value_cols] + np.random.normal(size=(data[value_cols].shape[0], data[value_cols].shape[1]))*(1.e-8)




In [26]:
corr_data = pd.DataFrame(data_zscore.groupby(by='PROLIFIC_PID').corr(numeric_only = True)).reset_index()
corr_data = corr_data[['PROLIFIC_PID','level_1'] + value_cols]
corr_data_var =  corr_data.groupby(by=['level_1']).var(numeric_only = True)


In [27]:
# Load packages
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor



In [28]:
# Load dataset
data_LR = data_zscore.copy()
data_LR = data_LR.merge(data[['PROLIFIC_PID','race','polit_party', 'wave']], on = ['PROLIFIC_PID', 'wave'], how = 'left')
data_LR.sex = data_LR.sex.str.replace('Female, Female','Female')

In [29]:
data_LR

,PROLIFIC_PID,state,county,loc_description,wave,wave_day,date,state_county,sex,agecats,...,cumsum_cases,cumsum_deaths,cases_avg,cases_avg_per_100k,deaths_avg,deaths_avg_per_100k,ueRate,restriction_sum,race,polit_party
0,546ec14dfdf99b2bc7ebd032,california,contra costa,suburb_large_city,1,start_date-1,2020-04-03,california_contra costa,Male,age55_59,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.512801,Asian,Republican
1,546ec14dfdf99b2bc7ebd032,california,contra costa,suburb_large_city,2,start_date-1,2020-04-10,california_contra costa,Male,age55_59,...,-0.319115,-0.296062,-0.323851,-0.663032,-0.210304,-0.544004,0.860941,0.400016,Asian,Republican
2,546ec14dfdf99b2bc7ebd032,california,contra costa,suburb_large_city,3,start_date-1,2020-04-17,california_contra costa,Male,age55_59,...,-0.317637,-0.292901,-0.336875,-0.705052,-0.204909,-0.508084,0.860941,0.400016,Asian,Republican
3,546ec14dfdf99b2bc7ebd032,california,contra costa,suburb_large_city,4,start_date-1,2020-04-24,california_contra costa,Male,age55_59,...,-0.316149,-0.291637,-0.336684,-0.704545,-0.197716,-0.472163,0.860941,0.400016,Asian,Republican
4,546ec14dfdf99b2bc7ebd032,california,contra costa,suburb_large_city,5,start_date-1,2020-05-08,california_contra costa,Male,age55_59,...,-0.313690,-0.289740,-0.342700,-0.723783,-0.194119,-0.454203,0.632840,0.400016,Asian,Republican
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18413,5e867394499bf02a03897f0a,illinois,cook,middle_large_city_dense,7,start_date-1,2020-06-05,illinois_cook,Female,age18_24,...,0.552711,0.927543,0.623750,-0.077801,1.656923,1.593271,1.213462,0.187481,Black or African American,Democrat
18414,5e867394499bf02a03897f0a,illinois,cook,middle_large_city_dense,8,start_date-1,2020-06-26,illinois_cook,Female,age18_24,...,0.636018,1.131741,0.125841,-0.437749,0.852787,0.659336,1.213462,-0.237590,Black or African American,Democrat
18415,5e867394499bf02a03897f0a,illinois,cook,middle_large_city_dense,9,start_date-1,2020-07-17,illinois_cook,Female,age18_24,...,0.730089,1.207288,0.255714,-0.344091,0.243465,-0.041116,0.715786,-0.237590,Black or African American,Democrat
18416,5e867394499bf02a03897f0a,illinois,cook,middle_large_city_dense,10,start_date-1,2020-08-07,illinois_cook,Female,age18_24,...,0.861802,1.256283,0.513121,-0.157789,0.038460,-0.274600,0.674313,-0.237590,Black or African American,Democrat


In [30]:
# R2 for mixed effects model
def get_R2(data, dep_var, modelfit):
    # Get the total variance of the model
    total_var = np.var(data[dep_var])

    # Get the variance of the fixed effects
    fixed_var = np.var(modelfit.fittedvalues)
    # Get the variance of the random effects
    random_effects_list = list(modelfit.random_effects.values())
    random_effects = np.empty((len(random_effects_list)))
    for idx, rand_eff in enumerate(random_effects_list):
        random_effects[idx] = rand_eff.values[0]
    random_var = np.mean(random_effects ** 2)

    # Calculate the marginal R-squared
    marginal_r2 = fixed_var / total_var

    # Calculate the conditional R-squared
    conditional_r2 = (fixed_var + random_var) / total_var
    
    return random_var, marginal_r2, conditional_r2


# EXTERNAL MEASURES

In [31]:
y_vec = ['cvd_concern','cvd_norm_agreement','restrict_f1']
x_var = 'ueRate'


for y in y_vec:
    data_LR_noNA = data_LR.dropna(subset=[y, x_var]).copy().reset_index(drop = True)
    data_LR_noNA = data_LR_noNA.drop_duplicates()
    data_LR_noNA = data_LR_noNA.rename(columns = {y:'y'})
    
    print(y)
 
    md = smf.mixedlm("y ~ ueRate",
                         data = data_LR_noNA, groups= "PROLIFIC_PID")
    mdf = md.fit(method=["lbfgs"])
    print(mdf.summary())
    random_var, marginal_r2, conditional_r2 = get_R2(data_LR_noNA, 'y', mdf)
    
    print('R-squared: ', marginal_r2)
    data_LR_noNA = data_LR_noNA.rename(columns = {'y':y})


cvd_concern
           Mixed Linear Model Regression Results
Model:              MixedLM Dependent Variable: y          
No. Observations:   16865   Method:             REML       
No. Groups:         1569    Scale:              0.2067     
Min. group size:    1       Log-Likelihood:     -13405.5105
Max. group size:    16      Converged:          Yes        
Mean group size:    10.7                                   
-----------------------------------------------------------
                 Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-----------------------------------------------------------
Intercept        -0.038    0.023 -1.668 0.095 -0.082  0.007
ueRate            0.040    0.005  8.473 0.000  0.031  0.050
PROLIFIC_PID Var  0.770    0.066                           

R-squared:  0.7769093676495129
cvd_norm_agreement
           Mixed Linear Model Regression Results
Model:              MixedLM Dependent Variable: y          
No. Observations:   16865   Method:             REML       

In [77]:
y_vec = ['cvd_concern','cvd_norm_agreement','restrict_f1']
x_var = 'deaths_avg_per_100k'


for y in y_vec:
    data_LR_noNA = data_LR.dropna(subset=[y, x_var]).copy().reset_index(drop = True)
    data_LR_noNA = data_LR_noNA.drop_duplicates()
    data_LR_noNA = data_LR_noNA.rename(columns = {y:'y'})
    
    print(y)
 
    md = smf.mixedlm("y ~ deaths_avg_per_100k",
                         data = data_LR_noNA, groups= "PROLIFIC_PID")
    mdf = md.fit(method=["lbfgs"])

    print(mdf.summary())
    random_var, marginal_r2, conditional_r2 = get_R2(data_LR_noNA, 'y', mdf)
    
    print('R-squared: ', marginal_r2)
    data_LR_noNA = data_LR_noNA.rename(columns = {'y':y})

cvd_concern
            Mixed Linear Model Regression Results
Model:               MixedLM  Dependent Variable:  y          
No. Observations:    16818    Method:              REML       
No. Groups:          1568     Scale:               0.2076     
Min. group size:     1        Log-Likelihood:      -13403.1021
Max. group size:     17       Converged:           Yes        
Mean group size:     10.7                                     
--------------------------------------------------------------
                    Coef.  Std.Err.   z    P>|z| [0.025 0.975]
--------------------------------------------------------------
Intercept           -0.033    0.023 -1.460 0.144 -0.077  0.011
deaths_avg_per_100k  0.009    0.004  2.073 0.038  0.000  0.017
PROLIFIC_PID Var     0.770    0.066                           

R-squared:  0.7760074937986602
cvd_norm_agreement
            Mixed Linear Model Regression Results
Model:               MixedLM  Dependent Variable:  y          
No. Observations: 

In [78]:
y_vec = ['cvd_concern','cvd_norm_agreement','restrict_f1']
x_var = 'cases_avg_per_100k'


for y in y_vec:
    data_LR_noNA = data_LR.dropna(subset=[y, x_var]).copy().reset_index(drop = True)
    data_LR_noNA = data_LR_noNA.drop_duplicates()
    data_LR_noNA = data_LR_noNA.rename(columns = {y:'y'})
    
    print(y)
 
    md = smf.mixedlm("y ~ cases_avg_per_100k",
                         data = data_LR_noNA, groups= "PROLIFIC_PID")
    mdf = md.fit(method=["lbfgs"])

    print(mdf.summary())
    random_var, marginal_r2, conditional_r2 = get_R2(data_LR_noNA, 'y', mdf)
    
    print('R-squared: ', marginal_r2)
    data_LR_noNA = data_LR_noNA.rename(columns = {'y':y})

cvd_concern
            Mixed Linear Model Regression Results
Model:              MixedLM  Dependent Variable:  y          
No. Observations:   16818    Method:              REML       
No. Groups:         1568     Scale:               0.2062     
Min. group size:    1        Log-Likelihood:      -13351.4965
Max. group size:    17       Converged:           Yes        
Mean group size:    10.7                                     
-------------------------------------------------------------
                   Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------
Intercept          -0.030    0.023 -1.341 0.180 -0.075  0.014
cases_avg_per_100k  0.039    0.004 10.400 0.000  0.032  0.046
PROLIFIC_PID Var    0.769    0.066                           

R-squared:  0.777496215260994
cvd_norm_agreement
            Mixed Linear Model Regression Results
Model:              MixedLM  Dependent Variable:  y          
No. Observations:   16818    Met

In [79]:
y_vec = ['cvd_concern','cvd_norm_agreement','restrict_f1']
x_var = 'restriction_sum'

#,'race':'White' , 'agecats':'age18_24','sex': 'male'}

for y in y_vec:
    data_LR_noNA = data_LR.dropna(subset=[y, x_var]).copy().reset_index(drop = True)
    data_LR_noNA = data_LR_noNA.drop_duplicates()
    data_LR_noNA = data_LR_noNA.rename(columns = {y:'y'})
    
    print(y)
 
    md = smf.mixedlm("y ~ restriction_sum",
                         data = data_LR_noNA, groups= "PROLIFIC_PID")
    mdf = md.fit(method=["lbfgs"])
    
    print(mdf.summary())
    random_var, marginal_r2, conditional_r2 = get_R2(data_LR_noNA, 'y', mdf)
    
    print('R-squared: ', marginal_r2)
    data_LR_noNA = data_LR_noNA.rename(columns = {'y':y})

cvd_concern
          Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: y          
No. Observations: 18400   Method:             REML       
No. Groups:       1568    Scale:              0.2202     
Min. group size:  1       Log-Likelihood:     -14990.3314
Max. group size:  18      Converged:          Yes        
Mean group size:  11.7                                   
---------------------------------------------------------
                 Coef. Std.Err.   z   P>|z| [0.025 0.975]
---------------------------------------------------------
Intercept        0.003    0.022 0.130 0.897 -0.041  0.047
restriction_sum  0.004    0.004 1.112 0.266 -0.003  0.012
PROLIFIC_PID Var 0.751    0.062                          

R-squared:  0.7617585734590926
cvd_norm_agreement
          Mixed Linear Model Regression Results
Model:             MixedLM Dependent Variable: y          
No. Observations:  18400   Method:             REML       
No. Groups:        1568    

# SELF REPORTED COVID IMPACT

In [151]:
y_vec = ['cvd_concern','cvd_norm_agreement','restrict_f1']
x_var = 'weekly_income_self_cat'


for y in y_vec:
    data_LR_noNA = data_LR.dropna(subset=[y, x_var]).copy().reset_index(drop = True)
    data_LR_noNA = data_LR_noNA.drop_duplicates()
    data_LR_noNA = data_LR_noNA.rename(columns = {y:'y'})
    
    print(y)
 
    md = smf.mixedlm("y ~ C(weekly_income_self_cat,Treatment('<$250'))",
                         data = data_LR_noNA, groups= "PROLIFIC_PID")
    mdf = md.fit(method=["lbfgs"])
    print(mdf.summary())
    random_var, marginal_r2, conditional_r2 = get_R2(data_LR_noNA, 'y', mdf)
    
    print('R-squared: ', marginal_r2)
    data_LR_noNA = data_LR_noNA.rename(columns = {'y':y})
    
    
tmp_df = data_LR.loc[data_LR.wave == 1,['PROLIFIC_PID', x_var]].copy().drop_duplicates()
print(tmp_df[x_var].value_counts())
sum(tmp_df[x_var].value_counts())

cvd_concern
                                  Mixed Linear Model Regression Results
Model:                            MixedLM                 Dependent Variable:                 y          
No. Observations:                 18400                   Method:                             REML       
No. Groups:                       1568                    Scale:                              0.2201     
Min. group size:                  1                       Log-Likelihood:                     -14999.0722
Max. group size:                  18                      Converged:                          Yes        
Mean group size:                  11.7                                                                   
---------------------------------------------------------------------------------------------------------
                                                               Coef.  Std.Err.   z    P>|z| [0.025 0.975]
--------------------------------------------------------------------

1537

In [133]:
y_vec = ['cvd_concern','cvd_norm_agreement','restrict_f1']
x_var = 'safety_measures'


for y in y_vec:
    data_LR_noNA = data_LR.dropna(subset=[y, x_var]).copy().reset_index(drop = True)
    data_LR_noNA = data_LR_noNA.drop_duplicates()
    data_LR_noNA = data_LR_noNA.rename(columns = {y:'y'})
    
    print(y)
 
    md = smf.mixedlm("y ~ safety_measures",
                         data = data_LR_noNA, groups= "PROLIFIC_PID")
    mdf = md.fit(method=["lbfgs"])
    print(mdf.summary())
    random_var, marginal_r2, conditional_r2 = get_R2(data_LR_noNA, 'y', mdf)
    
    print('R-squared: ', marginal_r2)
    data_LR_noNA = data_LR_noNA.rename(columns = {'y':y})

cvd_concern
           Mixed Linear Model Regression Results
Model:              MixedLM Dependent Variable: y          
No. Observations:   18400   Method:             REML       
No. Groups:         1568    Scale:              0.2144     
Min. group size:    1       Log-Likelihood:     -14686.9272
Max. group size:    18      Converged:          Yes        
Mean group size:    11.7                                   
-----------------------------------------------------------
                 Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-----------------------------------------------------------
Intercept        -0.003    0.021 -0.140 0.888 -0.044  0.039
safety_measures   0.154    0.006 24.812 0.000  0.142  0.167
PROLIFIC_PID Var  0.676    0.057                           

R-squared:  0.7680398084304323
cvd_norm_agreement
          Mixed Linear Model Regression Results
Model:             MixedLM Dependent Variable: y          
No. Observations:  18400   Method:             REML       
No

In [134]:
y_vec = ['cvd_concern','cvd_norm_agreement','restrict_f1']
x_var = ['spouse_health_impact', 
 'parent_health_impact', 
 'grandparent_health_impact',
 'thirdDegree_health_impact',  
 'child_health_impact', 
 'friend_health_impact']


for y in y_vec:
    data_LR_noNA = data_LR.dropna(subset=[y] + x_var).copy().reset_index(drop = True)
    data_LR_noNA = data_LR_noNA.drop_duplicates()
    data_LR_noNA = data_LR_noNA.rename(columns = {y:'y'})
    
    print(y)
 
    md = smf.mixedlm("y ~  C(spouse_health_impact,Treatment(0)) + C(parent_health_impact,Treatment(0))+ C(grandparent_health_impact,Treatment(0))+ C(thirdDegree_health_impact,Treatment(0))+ C(child_health_impact,Treatment(0))+ C(friend_health_impact,Treatment(0))",
                         data = data_LR_noNA, groups= "PROLIFIC_PID")
    mdf = md.fit(method=["lbfgs"])
    print(mdf.summary())
    random_var, marginal_r2, conditional_r2 = get_R2(data_LR_noNA, 'y', mdf)
    
    print('R-squared: ', marginal_r2)
    data_LR_noNA = data_LR_noNA.rename(columns = {'y':y})

cvd_concern
                           Mixed Linear Model Regression Results
Model:                         MixedLM            Dependent Variable:            y          
No. Observations:              18400              Method:                        REML       
No. Groups:                    1568               Scale:                         0.2198     
Min. group size:               1                  Log-Likelihood:                -15022.6936
Max. group size:               18                 Converged:                     Yes        
Mean group size:               11.7                                                         
--------------------------------------------------------------------------------------------
                                                  Coef.  Std.Err.   z    P>|z| [0.025 0.975]
--------------------------------------------------------------------------------------------
Intercept                                          0.012    0.023  0.509 0.611 -0.033 

                            Mixed Linear Model Regression Results
Model:                        MixedLM             Dependent Variable:             y          
No. Observations:             18400               Method:                         REML       
No. Groups:                   1568                Scale:                          0.3118     
Min. group size:              1                   Log-Likelihood:                 -17821.5795
Max. group size:              18                  Converged:                      Yes        
Mean group size:              11.7                                                           
---------------------------------------------------------------------------------------------
                                                  Coef.  Std.Err.    z    P>|z| [0.025 0.975]
---------------------------------------------------------------------------------------------
Intercept                                          0.072    0.021   3.410 0.001  0.031  

# DEMS

In [14]:
# political party affiliation
y_vec = ['cvd_concern','cvd_norm_agreement','restrict_f1']
x_var = 'polit_party'
for y in y_vec:
    data_LR_noNA = data_LR.dropna(subset=[y, x_var]).copy().reset_index(drop = True)
    data_LR_noNA = data_LR_noNA.drop_duplicates()
    data_LR_noNA = data_LR_noNA.rename(columns = {y:'y'})
    
    print(y)
 
    md = smf.mixedlm("y ~ C(polit_party,Treatment('Independent'))",
                         data = data_LR_noNA, groups= "PROLIFIC_PID")
    mdf = md.fit(reml=False)
    print(mdf.summary())
    print('AIC: ', mdf.aic)
    random_var, marginal_r2, conditional_r2 = get_R2(data_LR_noNA, 'y', mdf)
    
    print('R-squared: ', marginal_r2)
    data_LR_noNA = data_LR_noNA.rename(columns = {'y':y})




cvd_concern
                              Mixed Linear Model Regression Results
Model:                          MixedLM              Dependent Variable:              y          
No. Observations:               18400                Method:                          ML         
No. Groups:                     1568                 Scale:                           0.2203     
Min. group size:                1                    Log-Likelihood:                  -14969.9737
Max. group size:                18                   Converged:                       Yes        
Mean group size:                11.7                                                             
-------------------------------------------------------------------------------------------------
                                                       Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------------------------------------------
Intercept                             

In [19]:
data_LR.loc[data_LR.wave == 1]

,PROLIFIC_PID,state,county,loc_description,wave,wave_day,date,state_county,sex,agecats,...,cumsum_cases,cumsum_deaths,cases_avg,cases_avg_per_100k,deaths_avg,deaths_avg_per_100k,ueRate,restriction_sum,race,polit_party
0,546ec14dfdf99b2bc7ebd032,california,contra costa,suburb_large_city,1,start_date-1,2020-04-03,california_contra costa,Male,age55_59,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.512801,Asian,Republican
17,546ee363fdf99b2bc7ebd035,california,san diego,suburb_large_city,1,start_date-1,2020-04-03,california_san diego,Female,age55_59,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.512801,White,Democrat
33,5484620efdf99b0379939c6a,pennsylvania,centre,countryside_near_town,1,start_date-1,2020-04-03,pennsylvania_centre,Male,age55_59,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.512801,White,Independent
38,54847013fdf99b0379939c8a,michigan,kent,countryside_near_town,1,start_date-1,2020-04-03,michigan_kent,Female,age40_44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.512801,White,Republican
55,54847f1cfdf99b07b28f22f9,missouri,jackson,middle_med_city,1,start_date-1,2020-04-03,missouri_jackson,Male,age45_49,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.512801,Black or African American,Democrat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18367,5e850b0e390e520ec806b084,illinois,cook,middle_large_city_spread,1,start_date-1,2020-04-03,illinois_cook,Male,age35_39,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.512801,Black or African American,Democrat
18383,5e85547fc0a35112943cc942,alabama,lee,suburb_small_city,1,start_date-1,2020-04-03,alabama_lee,Female,age35_39,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.512801,Black or African American,Democrat
18387,5e8639bd69ef02096abc2ba8,california,los angeles,suburb_med_city,1,start_date-1,2020-04-03,california_los angeles,Male,age35_39,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.512801,Asian,Other
18391,5e86543a53859c0cad6cfcaa,florida,miami-dade,suburb_large_city,1,start_date-1,2020-04-03,florida_miami-dade,Male,age25_29,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.512801,Black or African American,Democrat


In [147]:
# race
y_vec = ['cvd_concern','cvd_norm_agreement','restrict_f1']
x_var ='race'
for y in y_vec:
    data_LR_noNA = data_LR.dropna(subset=[y, x_var]).copy().reset_index(drop = True)
    data_LR_noNA = data_LR_noNA.drop_duplicates()
    data_LR_noNA = data_LR_noNA.rename(columns = {y:'y'})
    
    print(y)
 
    md = smf.mixedlm("y ~ C(race,Treatment('White'))",
                         data = data_LR_noNA, groups= "PROLIFIC_PID")
    mdf = md.fit(reml=False)
    print(mdf.summary())
    print('AIC: ', mdf.aic)
    random_var, marginal_r2, conditional_r2 = get_R2(data_LR_noNA, 'y', mdf)
    
    print('R-squared: ', marginal_r2)
    data_LR_noNA = data_LR_noNA.rename(columns = {'y':y})
    
tmp_df = data_LR[['PROLIFIC_PID', x_var]].copy().drop_duplicates()
print(tmp_df[x_var].value_counts())
sum(tmp_df[x_var].value_counts())

cvd_concern
                                       Mixed Linear Model Regression Results
Model:                                MixedLM                    Dependent Variable:                    y          
No. Observations:                     18400                      Method:                                ML         
No. Groups:                           1568                       Scale:                                 0.2202     
Min. group size:                      1                          Log-Likelihood:                        -14982.7883
Max. group size:                      18                         Converged:                             Yes        
Mean group size:                      11.7                                                                         
-------------------------------------------------------------------------------------------------------------------
                                                                         Coef.  Std.Err.   z    P>|

1568

In [148]:
# age
y_vec = ['cvd_concern','cvd_norm_agreement','restrict_f1']

x_var = 'agecats'
for y in y_vec:
    data_LR_noNA = data_LR.dropna(subset=[y, x_var]).copy().reset_index(drop = True)
    data_LR_noNA = data_LR_noNA.drop_duplicates()
    data_LR_noNA = data_LR_noNA.rename(columns = {y:'y'})
    
    print(y)
 
    md = smf.mixedlm("y ~ C(agecats,Treatment('age18_24'))",
                         data = data_LR_noNA, groups= "PROLIFIC_PID")
    mdf = md.fit(reml=False)
    print(mdf.summary())
    print('AIC: ', mdf.aic)
    random_var, marginal_r2, conditional_r2 = get_R2(data_LR_noNA, 'y', mdf)
    
    print('R-squared: ', marginal_r2)
    #data_LR_noNA = data_LR_noNA.rename(columns = {'y':y})
    
tmp_df = data_LR[['PROLIFIC_PID', x_var]].copy().drop_duplicates()
print(tmp_df[x_var].value_counts())
sum(tmp_df[x_var].value_counts())

cvd_concern
                          Mixed Linear Model Regression Results
Model:                        MixedLM           Dependent Variable:           y          
No. Observations:             18400             Method:                       ML         
No. Groups:                   1568              Scale:                        0.2202     
Min. group size:              1                 Log-Likelihood:               -14977.2364
Max. group size:              18                Converged:                    Yes        
Mean group size:              11.7                                                       
-----------------------------------------------------------------------------------------
                                               Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-----------------------------------------------------------------------------------------
Intercept                                      -0.138    0.055 -2.490 0.013 -0.247 -0.029
C(agecats, Treatment('ag

1568

In [149]:
# sex
y_vec = ['cvd_concern','cvd_norm_agreement','restrict_f1']
x_var = 'sex'
for y in y_vec:
    data_LR_noNA = data_LR.dropna(subset=[y, x_var]).copy().reset_index(drop = True)
    data_LR_noNA = data_LR_noNA.drop_duplicates()
    data_LR_noNA = data_LR_noNA.rename(columns = {y:'y'})
    
    print(y)
 
    md = smf.mixedlm("y ~ C(sex,Treatment('Male'))",
                         data = data_LR_noNA, groups= "PROLIFIC_PID")
    mdf = md.fit(method=["lbfgs"])
    print(mdf.summary())
    print('AIC: ', mdf.aic)
    random_var, marginal_r2, conditional_r2 = get_R2(data_LR_noNA, 'y', mdf)
    
    print('R-squared: ', marginal_r2)
    data_LR_noNA = data_LR_noNA.rename(columns = {'y':y})

tmp_df = data_LR[['PROLIFIC_PID', x_var]].copy().drop_duplicates()
print(tmp_df[x_var].value_counts())
sum(tmp_df[x_var].value_counts())

cvd_concern
                          Mixed Linear Model Regression Results
Model:                        MixedLM           Dependent Variable:           y          
No. Observations:             18400             Method:                       REML       
No. Groups:                   1568              Scale:                        0.2202     
Min. group size:              1                 Log-Likelihood:               -14972.5748
Max. group size:              18                Converged:                    Yes        
Mean group size:              11.7                                                       
-----------------------------------------------------------------------------------------
                                               Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-----------------------------------------------------------------------------------------
Intercept                                      -0.097    0.032 -3.062 0.002 -0.159 -0.035
C(sex, Treatment('Male')

1568

# UE RATE X DEMS

'polit_party': 'Independent'

In [47]:
y_vec = ['cvd_concern','cvd_norm_agreement','restrict_f1']

for y in y_vec:
    data_LR_noNA = data_LR.dropna(subset=[y, x_var]).copy().reset_index(drop = True)
    data_LR_noNA = data_LR_noNA.drop_duplicates()
    data_LR_noNA = data_LR_noNA.rename(columns = {y:'y'})
    
    print(y)
 
    md = smf.mixedlm("y ~ ueRate + C(polit_party,Treatment('Independent'))",
                         data = data_LR_noNA, groups= "PROLIFIC_PID")
    mdf = md.fit(method=["lbfgs"])
    print(mdf.summary())
    data_LR_noNA = data_LR_noNA.rename(columns = {'y':y})


vif = [variance_inflation_factor(mdf.model.exog, i) for i in range(mdf.model.exog.shape[1])]
print(vif)

cvd_concern
                              Mixed Linear Model Regression Results
Model:                          MixedLM              Dependent Variable:              y          
No. Observations:               16863                Method:                          REML       
No. Groups:                     1568                 Scale:                           0.2070     
Min. group size:                1                    Log-Likelihood:                  -13400.5993
Max. group size:                17                   Converged:                       Yes        
Mean group size:                10.8                                                             
-------------------------------------------------------------------------------------------------
                                                       Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------------------------------------------
Intercept                             

'race':'White' 

In [48]:
y_vec = ['cvd_concern','cvd_norm_agreement','restrict_f1']



for y in y_vec:
    data_LR_noNA = data_LR.dropna(subset=[y, x_var]).copy().reset_index(drop = True)
    data_LR_noNA = data_LR_noNA.drop_duplicates()
    data_LR_noNA = data_LR_noNA.rename(columns = {y:'y'})
    
    print(y)
 
    md = smf.mixedlm("y ~ ueRate + C(race,Treatment('White'))",
                         data = data_LR_noNA, groups= "PROLIFIC_PID")
    mdf = md.fit(method=["lbfgs"])
    print(mdf.summary())
    data_LR_noNA = data_LR_noNA.rename(columns = {'y':y})


vif = [variance_inflation_factor(mdf.model.exog, i) for i in range(mdf.model.exog.shape[1])]
print(vif)

cvd_concern
                                       Mixed Linear Model Regression Results
Model:                                MixedLM                    Dependent Variable:                    y          
No. Observations:                     16863                      Method:                                REML       
No. Groups:                           1568                       Scale:                                 0.2068     
Min. group size:                      1                          Log-Likelihood:                        -13410.5551
Max. group size:                      17                         Converged:                             Yes        
Mean group size:                      10.8                                                                         
-------------------------------------------------------------------------------------------------------------------
                                                                         Coef.  Std.Err.   z    P>|

'agecats':'age18_24'

In [49]:
y_vec = ['cvd_concern','cvd_norm_agreement','restrict_f1']



for y in y_vec:
    data_LR_noNA = data_LR.dropna(subset=[y, x_var]).copy().reset_index(drop = True)
    data_LR_noNA = data_LR_noNA.drop_duplicates()
    data_LR_noNA = data_LR_noNA.rename(columns = {y:'y'})
    
    print(y)
 
    md = smf.mixedlm("y ~ ueRate + C(agecats,Treatment('age18_24'))",
                         data = data_LR_noNA, groups= "PROLIFIC_PID")
    mdf = md.fit(method=["lbfgs"])
    print(mdf.summary())
    data_LR_noNA = data_LR_noNA.rename(columns = {'y':y})


vif = [variance_inflation_factor(mdf.model.exog, i) for i in range(mdf.model.exog.shape[1])]
print(vif)

cvd_concern
                          Mixed Linear Model Regression Results
Model:                        MixedLM           Dependent Variable:           y          
No. Observations:             16863             Method:                       REML       
No. Groups:                   1568              Scale:                        0.2068     
Min. group size:              1                 Log-Likelihood:               -13411.9870
Max. group size:              17                Converged:                    Yes        
Mean group size:              10.8                                                       
-----------------------------------------------------------------------------------------
                                               Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-----------------------------------------------------------------------------------------
Intercept                                      -0.184    0.056 -3.255 0.001 -0.295 -0.073
C(agecats, Treatment('ag

'sex': 'male'

In [50]:
y_vec = ['cvd_concern','cvd_norm_agreement','restrict_f1']



for y in y_vec:
    data_LR_noNA = data_LR.dropna(subset=[y, x_var]).copy().reset_index(drop = True)
    data_LR_noNA = data_LR_noNA.drop_duplicates()
    data_LR_noNA = data_LR_noNA.rename(columns = {y:'y'})
    
    print(y)
 
    md = smf.mixedlm("y ~ ueRate + C(sex,Treatment('Male'))",
                         data = data_LR_noNA, groups= "PROLIFIC_PID")
    mdf = md.fit(method=["lbfgs"])
    print(mdf.summary())
    data_LR_noNA = data_LR_noNA.rename(columns = {'y':y})


vif = [variance_inflation_factor(mdf.model.exog, i) for i in range(mdf.model.exog.shape[1])]
print(vif)

cvd_concern
                          Mixed Linear Model Regression Results
Model:                        MixedLM           Dependent Variable:           y          
No. Observations:             16863             Method:                       REML       
No. Groups:                   1568              Scale:                        0.2068     
Min. group size:              1                 Log-Likelihood:               -13392.7965
Max. group size:              17                Converged:                    Yes        
Mean group size:              10.8                                                       
-----------------------------------------------------------------------------------------
                                               Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-----------------------------------------------------------------------------------------
Intercept                                      -0.136    0.032 -4.221 0.000 -0.198 -0.073
C(sex, Treatment('Male')

# COVID CASE COUNT X DEMS

'polit_party': 'Independent'

In [51]:
y_vec = ['cvd_concern','cvd_norm_agreement','restrict_f1']
x_var = 'cases_avg_per_100k'


for y in y_vec:
    data_LR_noNA = data_LR.dropna(subset=[y, x_var]).copy().reset_index(drop = True)
    data_LR_noNA = data_LR_noNA.drop_duplicates()
    data_LR_noNA = data_LR_noNA.rename(columns = {y:'y'})
    
    print(y)
 
    md = smf.mixedlm("y ~ cases_avg_per_100k + C(polit_party,Treatment('Independent'))",
                         data = data_LR_noNA, groups= "PROLIFIC_PID")
    mdf = md.fit(method=["lbfgs"])
    print(mdf.summary())
    data_LR_noNA = data_LR_noNA.rename(columns = {'y':y})


vif = [variance_inflation_factor(mdf.model.exog, i) for i in range(mdf.model.exog.shape[1])]
print(vif)

cvd_concern
                              Mixed Linear Model Regression Results
Model:                          MixedLM              Dependent Variable:              y          
No. Observations:               16818                Method:                          REML       
No. Groups:                     1568                 Scale:                           0.2064     
Min. group size:                1                    Log-Likelihood:                  -13346.7233
Max. group size:                17                   Converged:                       Yes        
Mean group size:                10.7                                                             
-------------------------------------------------------------------------------------------------
                                                       Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------------------------------------------
Intercept                             

'race':'White' 

In [53]:
y_vec = ['cvd_concern','cvd_norm_agreement','restrict_f1']
x_var = 'cases_avg_per_100k'


for y in y_vec:
    data_LR_noNA = data_LR.dropna(subset=[y, x_var]).copy().reset_index(drop = True)
    data_LR_noNA = data_LR_noNA.drop_duplicates()
    data_LR_noNA = data_LR_noNA.rename(columns = {y:'y'})
    
    print(y)
 
    md = smf.mixedlm("y ~ cases_avg_per_100k + C(race,Treatment('White'))",
                         data = data_LR_noNA, groups= "PROLIFIC_PID")
    mdf = md.fit(method=["lbfgs"])
    print(mdf.summary())
    data_LR_noNA = data_LR_noNA.rename(columns = {'y':y})


vif = [variance_inflation_factor(mdf.model.exog, i) for i in range(mdf.model.exog.shape[1])]
print(vif)

cvd_concern
                                       Mixed Linear Model Regression Results
Model:                                MixedLM                    Dependent Variable:                    y          
No. Observations:                     16818                      Method:                                REML       
No. Groups:                           1568                       Scale:                                 0.2062     
Min. group size:                      1                          Log-Likelihood:                        -13356.1886
Max. group size:                      17                         Converged:                             Yes        
Mean group size:                      10.7                                                                         
-------------------------------------------------------------------------------------------------------------------
                                                                         Coef.  Std.Err.   z    P>|

'agecats':'age18_24'

In [54]:
y_vec = ['cvd_concern','cvd_norm_agreement','restrict_f1']
x_var = 'cases_avg_per_100k'


for y in y_vec:
    data_LR_noNA = data_LR.dropna(subset=[y, x_var]).copy().reset_index(drop = True)
    data_LR_noNA = data_LR_noNA.drop_duplicates()
    data_LR_noNA = data_LR_noNA.rename(columns = {y:'y'})
    
    print(y)
 
    md = smf.mixedlm("y ~ cases_avg_per_100k + C(agecats,Treatment('age18_24'))",
                         data = data_LR_noNA, groups= "PROLIFIC_PID")
    mdf = md.fit(method=["lbfgs"])
    print(mdf.summary())
    data_LR_noNA = data_LR_noNA.rename(columns = {'y':y})


vif = [variance_inflation_factor(mdf.model.exog, i) for i in range(mdf.model.exog.shape[1])]
print(vif)

cvd_concern
                          Mixed Linear Model Regression Results
Model:                        MixedLM           Dependent Variable:           y          
No. Observations:             16818             Method:                       REML       
No. Groups:                   1568              Scale:                        0.2062     
Min. group size:              1                 Log-Likelihood:               -13358.6032
Max. group size:              17                Converged:                    Yes        
Mean group size:              10.7                                                       
-----------------------------------------------------------------------------------------
                                               Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-----------------------------------------------------------------------------------------
Intercept                                      -0.167    0.056 -2.958 0.003 -0.278 -0.056
C(agecats, Treatment('ag

'sex': 'Male'

In [55]:
y_vec = ['cvd_concern','cvd_norm_agreement','restrict_f1']
x_var = 'cases_avg_per_100k'


for y in y_vec:
    data_LR_noNA = data_LR.dropna(subset=[y, x_var]).copy().reset_index(drop = True)
    data_LR_noNA = data_LR_noNA.drop_duplicates()
    data_LR_noNA = data_LR_noNA.rename(columns = {y:'y'})
    
    print(y)
 
    md = smf.mixedlm("y ~ cases_avg_per_100k + C(sex,Treatment('Male'))",
                         data = data_LR_noNA, groups= "PROLIFIC_PID")
    mdf = md.fit(method=["lbfgs"])
    print(mdf.summary())
    data_LR_noNA = data_LR_noNA.rename(columns = {'y':y})


vif = [variance_inflation_factor(mdf.model.exog, i) for i in range(mdf.model.exog.shape[1])]
print(vif)

cvd_concern
                          Mixed Linear Model Regression Results
Model:                        MixedLM           Dependent Variable:           y          
No. Observations:             16818             Method:                       REML       
No. Groups:                   1568              Scale:                        0.2062     
Min. group size:              1                 Log-Likelihood:               -13338.8949
Max. group size:              17                Converged:                    Yes        
Mean group size:              10.7                                                       
-----------------------------------------------------------------------------------------
                                               Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-----------------------------------------------------------------------------------------
Intercept                                      -0.125    0.032 -3.898 0.000 -0.188 -0.062
C(sex, Treatment('Male')

# RESTRICTIONS X DEMS

'polit_party': 'Independent'

In [23]:
y_vec = ['cvd_concern','cvd_norm_agreement','restrict_f1']
x_var = 'restriction_sum'

#,'race':'White' , 'agecats':'age18_24','sex': 'male'}

for y in y_vec:
    data_LR_noNA = data_LR.dropna(subset=[y, x_var]).copy().reset_index(drop = True)
    data_LR_noNA = data_LR_noNA.drop_duplicates()
    data_LR_noNA = data_LR_noNA.rename(columns = {y:'y'})
    
    print(y)
 
    md = smf.mixedlm("y ~ restriction_sum + C(polit_party,Treatment('Independent'))",
                         data = data_LR_noNA, groups= "PROLIFIC_PID")
    mdf = md.fit(method=["lbfgs"])
    print(mdf.summary())
    data_LR_noNA = data_LR_noNA.rename(columns = {'y':y})


vif = [variance_inflation_factor(mdf.model.exog, i) for i in range(mdf.model.exog.shape[1])]
print(vif)

cvd_concern
                              Mixed Linear Model Regression Results
Model:                          MixedLM              Dependent Variable:              y          
No. Observations:               18400                Method:                          REML       
No. Groups:                     1568                 Scale:                           0.2203     
Min. group size:                1                    Log-Likelihood:                  -14984.4012
Max. group size:                18                   Converged:                       Yes        
Mean group size:                11.7                                                             
-------------------------------------------------------------------------------------------------
                                                       Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-------------------------------------------------------------------------------------------------
Intercept                             

'race':'White' 

In [31]:
y_vec = ['cvd_concern','cvd_norm_agreement','restrict_f1']
x_var = 'restriction_sum'

#,, 'agecats':'age18_24','sex': 'male'}

for y in y_vec:
    data_LR_noNA = data_LR.dropna(subset=[y, x_var]).copy().reset_index(drop = True)
    data_LR_noNA = data_LR_noNA.drop_duplicates()
    data_LR_noNA = data_LR_noNA.rename(columns = {y:'y'})
    
    print(y)
 
    md = smf.mixedlm("y ~ restriction_sum + C(race,Treatment('White'))",
                         data = data_LR_noNA, groups= "PROLIFIC_PID")
    mdf = md.fit(method=["lbfgs"])
    print(mdf.summary())
    data_LR_noNA = data_LR_noNA.rename(columns = {'y':y})


vif = [variance_inflation_factor(mdf.model.exog, i) for i in range(mdf.model.exog.shape[1])]
print(vif)

cvd_concern
                                       Mixed Linear Model Regression Results
Model:                                MixedLM                    Dependent Variable:                    y          
No. Observations:                     18400                      Method:                                REML       
No. Groups:                           1568                       Scale:                                 0.2202     
Min. group size:                      1                          Log-Likelihood:                        -14995.3386
Max. group size:                      18                         Converged:                             Yes        
Mean group size:                      11.7                                                                         
-------------------------------------------------------------------------------------------------------------------
                                                                         Coef.  Std.Err.   z    P>|

'agecats':'age18_24'

In [32]:
y_vec = ['cvd_concern','cvd_norm_agreement','restrict_f1']
x_var = 'restriction_sum'

#,, 'agecats':'age18_24','sex': 'male'}

for y in y_vec:
    data_LR_noNA = data_LR.dropna(subset=[y, x_var]).copy().reset_index(drop = True)
    data_LR_noNA = data_LR_noNA.drop_duplicates()
    data_LR_noNA = data_LR_noNA.rename(columns = {y:'y'})
    
    print(y)
 
    md = smf.mixedlm("y ~ restriction_sum + C(agecats,Treatment('age18_24'))",
                         data = data_LR_noNA, groups= "PROLIFIC_PID")
    mdf = md.fit(method=["lbfgs"])
    print(mdf.summary())
    data_LR_noNA = data_LR_noNA.rename(columns = {'y':y})


vif = [variance_inflation_factor(mdf.model.exog, i) for i in range(mdf.model.exog.shape[1])]
print(vif)

cvd_concern
                          Mixed Linear Model Regression Results
Model:                        MixedLM           Dependent Variable:           y          
No. Observations:             18400             Method:                       REML       
No. Groups:                   1568              Scale:                        0.2202     
Min. group size:              1                 Log-Likelihood:               -14996.7318
Max. group size:              18                Converged:                    Yes        
Mean group size:              11.7                                                       
-----------------------------------------------------------------------------------------
                                               Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-----------------------------------------------------------------------------------------
Intercept                                      -0.138    0.056 -2.484 0.013 -0.247 -0.029
C(agecats, Treatment('ag

'sex': 'male'

In [36]:
y_vec = ['cvd_concern','cvd_norm_agreement','restrict_f1']
x_var = 'restriction_sum'

for y in y_vec:
    data_LR_noNA = data_LR.dropna(subset=[y, x_var]).copy().reset_index(drop = True)
    data_LR_noNA = data_LR_noNA.drop_duplicates()
    data_LR_noNA = data_LR_noNA.rename(columns = {y:'y'})
    
    print(y)
 
    md = smf.mixedlm("y ~ restriction_sum + C(sex,Treatment('Male'))",
                         data = data_LR_noNA, groups= "PROLIFIC_PID")
    mdf = md.fit(method=["lbfgs"])
    print(mdf.summary())
    data_LR_noNA = data_LR_noNA.rename(columns = {'y':y})


vif = [variance_inflation_factor(mdf.model.exog, i) for i in range(mdf.model.exog.shape[1])]
print(vif)

cvd_concern
                          Mixed Linear Model Regression Results
Model:                        MixedLM           Dependent Variable:           y          
No. Observations:             18400             Method:                       REML       
No. Groups:                   1568              Scale:                        0.2202     
Min. group size:              1                 Log-Likelihood:               -14976.5626
Max. group size:              18                Converged:                    Yes        
Mean group size:              11.7                                                       
-----------------------------------------------------------------------------------------
                                               Coef.  Std.Err.   z    P>|z| [0.025 0.975]
-----------------------------------------------------------------------------------------
Intercept                                      -0.097    0.032 -3.068 0.002 -0.159 -0.035
C(sex, Treatment('Male')

In [26]:
vif = [variance_inflation_factor(mdf.model.exog, i) for i in range(mdf.model.exog.shape[1])]
print(vif)

[3.5282893030195734, 1.3479312938095596, 1.086520411397016, 1.29978478145177, 1.0004365684410326]


In [34]:
y_vec = ['restrict_f1','cvd_concern','safety_measures','cvd_norm_agreement']
x_var = 'cases_avg_per_100k'
dems = {'polit_party': 'Independent','race':'White' , 'agecats':'age18_24','sex': 'male'}

for y in y_vec:
    data_LR_noNA = data_LR.dropna(subset=[y, x_var]).copy().reset_index(drop = True)
    data_LR_noNA = data_LR_noNA.drop_duplicates()
    data_LR_noNA = data_LR_noNA.rename(columns = {y:'y'})
    
    for i_dem in dems.keys():
        print(i_dem)
        print(y)
        data_LR_noNA['dem_var'] = data_LR_noNA[i_dem].copy()
        # self restrict F1 ~ restrictions | political party
        md = smf.mixedlm("y ~ cases_avg_per_100k + C(dem_var)",
                         data = data_LR_noNA, groups= "PROLIFIC_PID")
        mdf = md.fit(method=["lbfgs"])
        print(mdf.summary())
    data_LR_noNA = data_LR_noNA.rename(columns = {'y':y})




polit_party
restrict_f1
                Mixed Linear Model Regression Results
Model:                MixedLM     Dependent Variable:     y          
No. Observations:     16818       Method:                 REML       
No. Groups:           1568        Scale:                  0.2907     
Min. group size:      1           Log-Likelihood:         -15826.1037
Max. group size:      17          Converged:              Yes        
Mean group size:      10.7                                           
---------------------------------------------------------------------
                          Coef.  Std.Err.    z    P>|z| [0.025 0.975]
---------------------------------------------------------------------
Intercept                  0.137    0.024   5.652 0.000  0.089  0.184
C(dem_var)[T.Independent] -0.178    0.028  -6.269 0.000 -0.234 -0.122
C(dem_var)[T.Other]       -0.232    0.048  -4.794 0.000 -0.327 -0.137
C(dem_var)[T.Republican]  -0.408    0.044  -9.239 0.000 -0.494 -0.321
cases_avg_pe

              Mixed Linear Model Regression Results
Model:               MixedLM    Dependent Variable:    y          
No. Observations:    17345      Method:                REML       
No. Groups:          1568       Scale:                 0.2196     
Min. group size:     1          Log-Likelihood:        -14197.2554
Max. group size:     18         Converged:             Yes        
Mean group size:     11.1                                         
------------------------------------------------------------------
                        Coef.  Std.Err.   z    P>|z| [0.025 0.975]
------------------------------------------------------------------
Intercept               -0.159    0.056 -2.832 0.005 -0.269 -0.049
C(dem_var)[T.age25_29]   0.132    0.081  1.635 0.102 -0.026  0.290
C(dem_var)[T.age30_34]   0.074    0.093  0.804 0.422 -0.107  0.256
C(dem_var)[T.age35_39]   0.065    0.148  0.439 0.661 -0.225  0.355
C(dem_var)[T.age40_44]   0.180    0.088  2.047 0.041  0.008  0.353
C(dem_var)

               Mixed Linear Model Regression Results
Model:                 MixedLM    Dependent Variable:    y          
No. Observations:      16818      Method:                REML       
No. Groups:            1568       Scale:                 0.3280     
Min. group size:       1          Log-Likelihood:        -16738.0082
Max. group size:       17         Converged:             Yes        
Mean group size:       10.7                                         
--------------------------------------------------------------------
                          Coef.  Std.Err.   z    P>|z| [0.025 0.975]
--------------------------------------------------------------------
Intercept                  0.126    0.024  5.179 0.000  0.078  0.173
C(dem_var)[T.Independent] -0.237    0.029 -8.049 0.000 -0.294 -0.179
C(dem_var)[T.Other]       -0.161    0.051 -3.178 0.001 -0.261 -0.062
C(dem_var)[T.Republican]  -0.401    0.045 -8.874 0.000 -0.489 -0.312
cases_avg_per_100k        -0.004    0.005 -0.824 0

In [ ]:
y_vec = ['restrict_f1','restrict_f2','restrict_f2','cvd_concern','safety_measures','cvd_norm_agreement']
x_var = 'restriction_sum'
dems = {'polit_party': 'Independent','race':'White','agecat': 'age18_24','sex': 'male'}

for y in y_vec:
    data_LR_noNA = data_LR.dropna(subset=[y, x_var]).copy().reset_index(drop = True)
    data_LR_noNA = data_LR_noNA.drop_duplicates()
    data_LR_noNA = data_LR_noNA.rename(columns = {y:'y'})
    
    for i_dem in dems.keys():
        data_LR_noNA = data_LR_noNA.rename(columns = {i_dem:'dem'})
        # self restrict F1 ~ restrictions | political party
        md = smf.mixedlm("y ~ x + C(dem)",
                         data = data_LR_noNA, groups= "PROLIFIC_PID")
        mdf = md.fit(method=["lbfgs"])
        print(mdf.summary())
        data_LR_noNA = data_LR_noNA.rename(columns = {'dem':i_dem})
    data_LR_noNA = data_LR_noNA.rename(columns = {'y':y})



# # self restrict F1 ~ restrictions | race
# md = smf.mixedlm("restrict_f1 ~ restriction_sum + C(race, Treatment('White'))",
#                  data = data_LR_noNA, groups= "PROLIFIC_PID")
# mdf = md.fit(method=["lbfgs"])
# print(mdf.summary())

# # self restrict F1 ~ restrictions | age
# md = smf.mixedlm("restrict_f1 ~ restriction_sum + C(agecats, Treatment(''))",
#                  data = data_LR_noNA, groups= "PROLIFIC_PID")
# mdf = md.fit(method=["lbfgs"])
# print(mdf.summary())

# # self restrict F1 ~ restrictions | sex
# md = smf.mixedlm("restrict_f1 ~ restriction_sum + C(sex, Treatment('male'))",
#                  data = data_LR_noNA, groups= "PROLIFIC_PID")
# mdf = md.fit(method=["lbfgs"])
# print(mdf.summary())

In [ ]:
data_LR_noNA = data_LR.dropna(subset=['restrict_f1', 'restriction_sum']).copy().reset_index(drop = True)
data_LR_noNA = data_LR_noNA.drop_duplicates()

# self restrict F1 ~ restrictions | political party
md = smf.mixedlm("restrict_f1 ~ restriction_sum + C(polit_party, Treatment('Independent'))",
                 data = data_LR_noNA, groups= "PROLIFIC_PID")
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

# self restrict F1 ~ restrictions | race
md = smf.mixedlm("restrict_f1 ~ restriction_sum + C(race, Treatment('White'))",
                 data = data_LR_noNA, groups= "PROLIFIC_PID")
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

# self restrict F1 ~ restrictions | age
md = smf.mixedlm("restrict_f1 ~ restriction_sum + C(agecats, Treatment('age18_24'))",
                 data = data_LR_noNA, groups= "PROLIFIC_PID")
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

# self restrict F1 ~ restrictions | sex
md = smf.mixedlm("restrict_f1 ~ restriction_sum + C(sex, Treatment('male'))",
                 data = data_LR_noNA, groups= "PROLIFIC_PID")
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

In [ ]:
data_LR_noNA = data_LR.dropna(subset=['restrict_f2', 'restriction_sum']).copy().reset_index(drop = True)
data_LR_noNA = data_LR_noNA.drop_duplicates()

# self restrict F2 ~ restrictions | political party
md = smf.mixedlm("restrict_f2 ~ restriction_sum + C(polit_party, Treatment('Independent'))",
                 data = data_LR_noNA, groups= "PROLIFIC_PID")
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

# self restrict F2 ~ restrictions | race
md = smf.mixedlm("restrict_f2 ~ restriction_sum + C(race, Treatment('White'))",
                 data = data_LR_noNA, groups= "PROLIFIC_PID")
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

# self restrict F2 ~ restrictions | age
md = smf.mixedlm("restrict_f2 ~ restriction_sum + C(agecats, Treatment('age18_24'))",
                 data = data_LR_noNA, groups= "PROLIFIC_PID")
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

# self restrict F2 ~ restrictions | sex
md = smf.mixedlm("restrict_f2 ~ restriction_sum + C(sex, Treatment('male'))",
                 data = data_LR_noNA, groups= "PROLIFIC_PID")
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

In [ ]:
data_LR_noNA = data_LR.dropna(subset=['restrict_f3', 'restriction_sum']).copy().reset_index(drop = True)
data_LR_noNA = data_LR_noNA.drop_duplicates()

# self restrict F3 ~ restrictions | political party
md = smf.mixedlm("restrict_f3 ~ restriction_sum + C(polit_party, Treatment('Independent'))",
                 data = data_LR_noNA, groups= "PROLIFIC_PID")
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

# self restrict F3 ~ restrictions | race
md = smf.mixedlm("restrict_f3 ~ restriction_sum + C(race, Treatment('White'))",
                 data = data_LR_noNA, groups= "PROLIFIC_PID")
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

# self restrict F3 ~ restrictions | age
md = smf.mixedlm("restrict_f3 ~ restriction_sum + C(agecats, Treatment('age18_24'))",
                 data = data_LR_noNA, groups= "PROLIFIC_PID")
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

# self restrict F3 ~ restrictions | sex
md = smf.mixedlm("restrict_f3 ~ restriction_sum + C(sex, Treatment('male'))",
                 data = data_LR_noNA, groups= "PROLIFIC_PID")
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

In [ ]:
data_LR_noNA = data_LR.dropna(subset=['cvd_concern', 'restriction_sum']).copy().reset_index(drop = True)
data_LR_noNA = data_LR_noNA.drop_duplicates()

# COVID concern ~ restrictions | political party
md = smf.mixedlm("cvd_concern ~ restriction_sum + C(polit_party, Treatment('Independent'))",
                 data = data_LR_noNA, groups= "PROLIFIC_PID")
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

# COVID concern ~ restrictions | race
md = smf.mixedlm("cvd_concern ~ restriction_sum + C(race, Treatment('White'))",
                 data = data_LR_noNA, groups= "PROLIFIC_PID")
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

# COVID concern ~ restrictions | age
md = smf.mixedlm("cvd_concern ~ restriction_sum + C(agecats, Treatment('age18_24'))",
                 data = data_LR_noNA, groups= "PROLIFIC_PID")
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

# COVID concern~ restrictions | sex
md = smf.mixedlm("cvd_concern ~ restriction_sum + C(sex, Treatment('male'))",
                 data = data_LR_noNA, groups= "PROLIFIC_PID")
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

In [ ]:
data_LR_noNA = data_LR.dropna(subset=['safety_measures', 'restriction_sum']).copy().reset_index(drop = True)
data_LR_noNA = data_LR_noNA.drop_duplicates()

# COVID concern ~ covid cases | political party
md = smf.mixedlm("safety_measures ~ restriction_sum + C(polit_party, Treatment('Independent'))",
                 data = data_LR_noNA, groups= "PROLIFIC_PID")
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

# COVID concern ~ covid cases | race
md = smf.mixedlm("safety_measures ~ restriction_sum + C(race, Treatment('White'))",
                 data = data_LR_noNA, groups= "PROLIFIC_PID")
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())



data_LR_noNA = data_LR.dropna(subset=['cvd_concern', 'restriction_sum']).copy().reset_index(drop = True)
data_LR_noNA = data_LR_noNA.drop_duplicates()

# COVID concern ~ covid cases | political party
md = smf.mixedlm("cvd_concern ~ restriction_sum + C(polit_party, Treatment('Independent'))",
                 data = data_LR_noNA, groups= "PROLIFIC_PID")
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

# COVID concern ~ covid cases | race
md = smf.mixedlm("cvd_concern ~ restriction_sum + C(race, Treatment('White'))",
                 data = data_LR_noNA, groups= "PROLIFIC_PID")
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

In [ ]:
data_LR_noNA

In [ ]:
data_LR_noNA = data_LR.dropna(subset=['cvd_norm_agreement', 'restriction_sum']).copy().reset_index(drop = True)
data_LR_noNA = data_LR_noNA.drop_duplicates()

# COVID concern ~ restrictions | political party
md = smf.mixedlm("cvd_norm_agreement ~ restriction_sum + C(polit_party, Treatment('Independent'))",
                 data = data_LR_noNA, groups= "PROLIFIC_PID")
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

# COVID concern ~ restrictions | race
md = smf.mixedlm("cvd_norm_agreement ~ restriction_sum + C(race, Treatment('White'))",
                 data = data_LR_noNA, groups= "PROLIFIC_PID")
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

# COVID concern ~ restrictions | age
md = smf.mixedlm("cvd_norm_agreement ~ restriction_sum + C(race, Treatment('White'))",
                 data = data_LR_noNA, groups= "PROLIFIC_PID")
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

In [ ]:
data_LR_noNA = data_LR.dropna(subset=['safety_measures', 'cvd_cases_7d_avg_per_100k']).copy().reset_index(drop = True)
data_LR_noNA = data_LR_noNA.drop_duplicates()

# COVID concern ~ covid cases | political party
md = smf.mixedlm("safety_measures ~ cvd_cases_7d_avg_per_100k + C(polit_party, Treatment('Independent'))",
                 data = data_LR_noNA, groups= "PROLIFIC_PID")
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

# COVID concern ~ covid cases | race
md = smf.mixedlm("safety_measures ~ cvd_cases_7d_avg_per_100k + C(race, Treatment('White'))",
                 data = data_LR_noNA, groups= "PROLIFIC_PID")
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

In [ ]:
data_LR_noNA = data_LR.dropna(subset=['restrict_f1', 'cvd_cases_7d_avg_per_100k']).copy().reset_index(drop = True)
data_LR_noNA = data_LR_noNA.drop_duplicates()

# COVID concern ~ covid cases | political party
md = smf.mixedlm("restrict_f1 ~ cvd_cases_7d_avg_per_100k + C(polit_party, Treatment('Independent'))",
                 data = data_LR_noNA, groups= "PROLIFIC_PID")
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

# COVID concern ~ covid cases | race
md = smf.mixedlm("restrict_f1 ~ cvd_cases_7d_avg_per_100k + C(race, Treatment('White'))",
                 data = data_LR_noNA, groups= "PROLIFIC_PID")
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

In [ ]:
data_LR_noNA = data_LR.dropna(subset=['restrict_f2', 'cvd_cases_7d_avg_per_100k']).copy().reset_index(drop = True)
data_LR_noNA = data_LR_noNA.drop_duplicates()

# COVID concern ~ covid cases | political party
md = smf.mixedlm("restrict_f2 ~ cvd_cases_7d_avg_per_100k + C(polit_party, Treatment('Independent'))",
                 data = data_LR_noNA, groups= "PROLIFIC_PID")
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

# COVID concern ~ covid cases | race
md = smf.mixedlm("restrict_f2 ~ cvd_cases_7d_avg_per_100k + C(race, Treatment('White'))",
                 data = data_LR_noNA, groups= "PROLIFIC_PID")
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

In [ ]:
data_LR_noNA = data_LR.dropna(subset=['restrict_f3', 'cvd_cases_7d_avg_per_100k']).copy().reset_index(drop = True)
data_LR_noNA = data_LR_noNA.drop_duplicates()

# COVID concern ~ covid cases | political party
md = smf.mixedlm("restrict_f3 ~ cvd_cases_7d_avg_per_100k + C(polit_party, Treatment('Independent'))",
                 data = data_LR_noNA, groups= "PROLIFIC_PID")
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

# COVID concern ~ covid cases | race
md = smf.mixedlm("restrict_f3 ~ cvd_cases_7d_avg_per_100k + C(race, Treatment('White'))",
                 data = data_LR_noNA, groups= "PROLIFIC_PID")
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

In [ ]:
data_LR_noNA = data_LR.dropna(subset=['cvd_norm_agreement', 'cvd_cases_7d_avg_per_100k']).copy().reset_index(drop = True)
data_LR_noNA = data_LR_noNA.drop_duplicates()

# COVID concern ~ covid cases | political party
md = smf.mixedlm("cvd_norm_agreement ~ cvd_cases_7d_avg_per_100k + C(polit_party, Treatment('Independent'))",
                 data = data_LR_noNA, groups= "PROLIFIC_PID")
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

# COVID concern ~ covid cases | race
md = smf.mixedlm("cvd_norm_agreement ~ cvd_cases_7d_avg_per_100k + C(race, Treatment('White'))",
                 data = data_LR_noNA, groups= "PROLIFIC_PID")
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

In [ ]:
data_LR_noNA = data_LR.dropna(subset=['cvd_concern', 'cvd_cases_7d_avg_per_100k']).copy().reset_index(drop = True)
data_LR_noNA = data_LR_noNA.drop_duplicates()

# COVID concern ~ covid cases | political party
md = smf.mixedlm("cvd_concern ~ cvd_cases_7d_avg_per_100k + C(polit_party, Treatment('Independent'))",
                 data = data_LR_noNA, groups= "PROLIFIC_PID")
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

# COVID concern ~ covid cases | race
md = smf.mixedlm("cvd_concern ~ cvd_cases_7d_avg_per_100k + C(race, Treatment('White'))",
                 data = data_LR_noNA, groups= "PROLIFIC_PID")
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

In [ ]:
data_LR_noNA = data_LR.dropna(subset=['cvd_concern', 'cvd_cases_7d_avg_per_100k']).copy().reset_index(drop = True)
data_LR_noNA = data_LR_noNA.drop_duplicates()

# COVID concern ~ covid cases | political party
md = smf.mixedlm("cvd_concern ~ cvd_cases_7d_avg_per_100k + C(polit_party, Treatment('Independent'))",
                 data = data_LR_noNA, groups= "PROLIFIC_PID")
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

# COVID concern ~ covid cases | race
md = smf.mixedlm("cvd_concern ~ cvd_cases_7d_avg_per_100k + C(race, Treatment('White'))",
                 data = data_LR_noNA, groups= "PROLIFIC_PID")
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())

In [ ]:
data_LR_noNA = data_LR.dropna(subset=['safety_measures', 'cvd_cases_7d_avg_per_100k']).copy().reset_index(drop = True)
data_LR_noNA = data_LR_noNA.drop_duplicates()

# Run LMER

md = smf.mixedlm("safety_measures ~ cvd_cases_7d_avg_per_100k", data = data_LR_noNA, groups= "PROLIFIC_PID")
mdf = md.fit(method=["lbfgs"])
mdf.summary()



In [ ]:
data_LR_noNA = data_LR.dropna(subset=['cvd_norm_agreement', 'cvd_cases_7d_avg_per_100k']).copy().reset_index(drop = True)
data_LR_noNA = data_LR_noNA.drop_duplicates()

# Run LMER

md = smf.mixedlm("cvd_norm_agreement ~ cvd_cases_7d_avg_per_100k + C(polit_party)", data = data_LR_noNA, groups= "PROLIFIC_PID")
mdf = md.fit(method=["lbfgs"])
mdf.summary()

In [ ]:
from scipy import stats


In [ ]:
data_noNA = data.dropna(subset=['cvd_concern', 'cvd_cases_7d_avg_per_100k']).copy().reset_index(drop = True)
plt.hist(data_noNA['cvd_cases_7d_avg_per_100k'],100);

In [ ]:
race, plitical identoy, income, age, sex 
BDI, NEO

In [ ]:
rand_noise = np.random.normal(size=(1,len(data_noNA)))*(1.e-16)
data_noNA['cvd_cases_7d_avg_per_100k']= data_noNA['cvd_cases_7d_avg_per_100k']+rand_noise.T.squeeze()


In [ ]:
plt.hist(data_noNA['cvd_cases_7d_avg_per_100k'],100);

In [ ]:
plt.hist(data_noNA['cvd_cases_7d_avg_per_100k'],100);

In [ ]:
subset = data.PROLIFIC_PID.unique()[:20]
data_LR_subset = data.loc[data.PROLIFIC_PID.isin(subset)]
sns.lmplot(data=data_LR_subset, x="cvd_cases_7d_avg_per_100k",  y="cvd_concern",hue="PROLIFIC_PID", legend = False)
ax = plt.gca()
# ax.set_xlim(-1, 3)
ax.set_ylim(0,8)

In [ ]:
data_LR_noNA = data_LR.dropna(subset=['cvd_concern', 'cvd_deaths_7d_avg_per_100k']).copy().reset_index(drop = True)
# Run LMER
md = smf.mixedlm("cvd_concern ~ cvd_deaths_7d_avg_per_100k", data_LR_noNA, groups=data_LR_noNA["PROLIFIC_PID"],re_formula="~cvd_deaths_7d_avg_per_100k")
mdf = md.fit(method=["lbfgs"])
print(mdf.summary())


subs = mdf.random_effects_cov.keys()
sub_random_effects = pd.DataFrame(index = subs)
for s in subs:
    sub_random_effects.loc[s, 'cvd_deaths_7d_avg_per_100k'] = mdf.random_effects[s]['cvd_deaths_7d_avg_per_100k']
sns.stripplot(sub_random_effects)